# 1. Preamboli

In [1]:
import pandas as pd

import mlflow
from mlflow.tracking import MlflowClient

from utils import load_dataset, train_and_log_model

from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient, dsl, command, Input, Output
from datetime import datetime
from zoneinfo import ZoneInfo

In [2]:
from config.secret import (
                             SUBSCRIPTION_ID
                            ,RESOURCE_GROUP_NAME
                            ,WORKSPACE_NAME
                            ,ENVIRONMENT_NAME
                            ,ENVIRONMENT_VERSION
                            ,COMPUTE_NAME
                            ,DATASTORE_NAME
                            ,BLOB_ACCOUNT_URL
                            ,BLOB_CONTAINER_NAME
                            ,MANAGED_IDENTITY_CLIENT_ID
                        )

DATASTORE_URI = f"azureml://subscriptions/{SUBSCRIPTION_ID}/resourcegroups/{RESOURCE_GROUP_NAME}/workspaces/{WORKSPACE_NAME}/datastores/{DATASTORE_NAME}/paths/"

In [3]:
from config.config import DEFAULT_INPUT_FILE_PATH

# 2. Autenticazione e preparazione environment

In [4]:
credential = DefaultAzureCredential(
    managed_identity_client_id=MANAGED_IDENTITY_CLIENT_ID
)

ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION_ID,
    resource_group_name=RESOURCE_GROUP_NAME,
    workspace_name=WORKSPACE_NAME,)

In [5]:
env = ml_client.environments.get( name = ENVIRONMENT_NAME,
                                  version = ENVIRONMENT_VERSION
                                  )

# 3. Creazione di una Componente per ogni step del ciclo di training

In [8]:
create_command = command(
    name='training_create_data',
    command="python -m components.01_create_data --input_file_path ${{inputs.input_file_path}} --experiment_name ${{inputs.experiment_name}} --data_asset_name ${{inputs.data_asset_name}} --create_output_path ${{outputs.create_output_path}}",
    environment=env,
    code='./',
    inputs={'input_file_path': Input(type='string'),
            'experiment_name': Input(type='string'), 
            'data_asset_name': Input(type='string')},
    outputs={'create_output_path': Output(type='uri_folder', mode='ro_mount')}
    )
create_component = ml_client.create_or_update(create_command.component)

Uploading research-mlflow (0.15 MBs): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150971/150971 [00:00<00:00, 391522.45it/s]


# 4. Definizione della pipeline (job) ed esecuzione

In [9]:
@dsl.pipeline(default_compute="cc-azml-01-dev")
def training_pipeline_job(experiment_name, timestamp):
    create_job = create_component(input_file_path = DEFAULT_INPUT_FILE_PATH,
                                  experiment_name = experiment_name,
                                  data_asset_name = 'raw_forecasting_dataset')
    # clean_job = clean_component(create_output_path = create_job.outputs.create_output_path,
    #                             clean_file_path = DEFAULT_CLEAN_FILE_PATH,
    #                             clean_data_asset_name = 'forecasting_dataset',
    #                             target_column = 'y',
    #                             date_column = 'date')
    # train_job = train_component(clean_output_path = clean_job.outputs.clean_output_path,
    #                             model_name = 'ForecastModel')
    # deploy_job = deploy_component(train_output_path = train_job.outputs.train_output_path,
    #                               endpoint_name = DEFAULT_ENDPOINT_NAME,
    #                               deployment_name = DEFAULT_DEPLOYMENT_NAME,
    #                               environment_name = env_name,
    #                               environment_version = str(env_version),
    #                               compute = 'cc1-dev03')
    # inference_job = inference_component(deploy_output_path = deploy_job.outputs.deploy_output_path,
    #                                     input_data_asset_name = 'raw_forecasting_dataset',
    #                                     output_data_path = f'batch_output_{timestamp}/{DEFAULT_OUTPUT_DATA_PATH}',
    #                                     output_data_asset_name = 'forecasting_prediction')

experiment_name = 'training_experiment_0'
timestamp = datetime.now(tz=ZoneInfo("Europe/Rome")).strftime("%Y%m%d_%H%M%S")
pipeline = training_pipeline_job(experiment_name, timestamp)
pipeline_job = ml_client.jobs.create_or_update(pipeline,
                                               experiment_name=experiment_name
                                               )

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


# 2. Import e versioning del dataset
Viene importato il dataset di esempio (iris oppure breast cancer, in base al parametro settato).

Questo dataset viene salvato in locale e versionato.

In [6]:
# TODO: Parametro da settare per scegliere il dataset
example_name = "breast_cancer" # "breast_cancer" / "iris"

- *breast cancer dataset:* problema di classificazione binaria (benigno/maligno), 30 features, 569 campioni
- *iris dataset:* problema di classificazione multi-classe (setosa/versicolor/virginica), 4 features, 150 campioni

In [7]:
dataset_name = f"{example_name}_dataset"

X_train, X_test, y_train, y_test, dataset_version, dataset_path, df = load_dataset(dataset_name)

print(f"Dataset salvato in: {dataset_path}")
print(f"Versione del dataset: {dataset_version}")

df.head(2)

Dataset salvato in: datasets/breast_cancer_dataset/v01.csv
Versione del dataset: 1


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0


# 3. Setup MLflow Tracking

In [ ]:
# Per impostare l'URI di tracciamento
# mlflow.set_tracking_uri("")

Lasciando il paramtro di default (in mlflow.set_tracking_uri), i dati vengono salvati in locale nella cartella "mlruns"

In [4]:
# Impostiamo il nome dell'esperimento
mlflow.set_experiment(f"{example_name}_Classification")

UnsupportedModelRegistryStoreURIException:  Model registry functionality is unavailable; got unsupported URI 'azureml://westeurope.api.azureml.ms/mlflow/v1.0/subscriptions/30101379-480a-4fdc-88d3-c30601a89b44/resourceGroups/rg-azml-demo-dev/providers/Microsoft.MachineLearningServices/workspaces/mlw-azml-demo-dev' for model registry data storage. Supported URI schemes are: ['', 'file', 'databricks', 'databricks-uc', 'uc', 'http', 'https', 'postgresql', 'mysql', 'sqlite', 'mssql']. See https://www.mlflow.org/docs/latest/tracking.html#storage for how to run an MLflow server against one of the supported backend storage locations.

L'esperimento risulta come una sorta di "cartella" sulla UI mlflow per suddividere il tracking di varie runs

## Runs

Vengono effettuate 4 runs diverse per sperimentare 4 diverse combinazioni di parametri nel training del modello.

In [ ]:
D_randomforest_1 = {
    "model": "RandomForest",
    "n_estimators": 10, 
    "max_depth": 3,
    }

D_randomforest_2= {
    "model": "RandomForest",
    "n_estimators": 20, 
    "max_depth": 5,
    }

D_logistic_1 = {
    "model": "LogisticRegression",
    "max_iter": 10
    }

D_logistic_2 = {
    "model": "LogisticRegression",
    "max_iter": 15
    }

D_list = [D_randomforest_1, D_randomforest_2, D_logistic_1, D_logistic_2]

In [ ]:
MODEL_NAME = f"{example_name}_Classifier"

In [ ]:
for D in D_list:
    # Eseguiamo alcuni esperimenti con parametri diversi
    run_id = train_and_log_model(   model_name = MODEL_NAME,
                                    model_dict = D,
                                    X_train = X_train,
                                    X_test = X_test,
                                    y_train = y_train,
                                    y_test = y_test,
                                    dataset_version = dataset_version,
                                    dataset_name = dataset_name,
                                    )

# 4. Gestione dei modelli e MLflow Model Registry

In questo esempio viene ricercato il modello migliore (con un accuracy più alta) tra tutti quelli registrati sotto "MODEL_NAME".

Successivamente, al modello migliore viene effettuato un cambio di stato, settandolo come modello "in produzione".

In [ ]:
client = MlflowClient()

# 1. Prendi tutte le versioni registrate del modello
versions = client.search_model_versions(f"name='{MODEL_NAME}'")

# 2. Trova l'accuracy migliore tra i run
best_run_id = None
best_accuracy = -1.0
best_model_version = None

for v in versions:
    run_id = v.run_id
    metrics = client.get_run(run_id).data.params
    acc = metrics.get("accuracy", None)
    if acc is not None and float(acc) > best_accuracy:
        best_accuracy = float(acc)
        best_run_id = run_id
        best_model_version = v.version

print(f"Miglior modello trovato: run_id={best_run_id}, versione={best_model_version}, accuracy={best_accuracy:.4f}")

In [ ]:
# 3. Aggiorna lo stato del modello migliore (es. in Production)
if best_model_version is not None:
    client.transition_model_version_stage(
        name=MODEL_NAME,
        version=best_model_version,
        stage="Production",   # oppure "Staging"
        archive_existing_versions=True  # sposta gli altri modelli fuori da Production
    )
    print(f"Il modello versione {best_model_version} è stato promosso a Production.")

# 5. Inference
Esempio di inferenza:

Utilizzando il modello che è stato messo in produzione, viene fatta inferenza utilizzando 3 nuovi record del dataset.

In [ ]:
# Prepara nuovi dati di input (stesso schema usato in addestramento)
if example_name == "iris":
    # Qui uso l'Iris dataset come esempio
    new_data = pd.DataFrame({
        "sepal length (cm)": [5.1, 6.2, 5.9],
        "sepal width (cm)":  [3.5, 2.9, 3.0],
        "petal length (cm)": [1.4, 4.3, 5.1],
        "petal width (cm)":  [0.2, 1.3, 1.8],
    })


elif example_name == "breast_cancer":
    new_data = pd.DataFrame({
        "mean radius": [14.0, 20.0, 13.5],
        "mean texture": [20.0, 30.0, 15.0],
        "mean perimeter": [90.0, 130.0, 85.0],
        "mean area": [600.0, 1200.0, 500.0],
        "mean smoothness": [0.1, 0.15, 0.09],
        "mean compactness": [0.1, 0.2, 0.08],
        "mean concavity": [0.05, 0.1, 0.04],
        "mean concave points": [0.02, 0.05, 0.01],
        "mean symmetry": [0.2, 0.3, 0.18],
        "mean fractal dimension": [0.06, 0.07, 0.05],
        "radius error": [0.3, 0.4, 0.2],
        "texture error": [1.5, 2.0, 1.2],
        "perimeter error": [2.5, 3.5, 2.0],
        "area error": [20.0, 30.0, 15.0],
        "smoothness error": [0.005, 0.007, 0.004],
        "compactness error": [0.02, 0.03, 0.015],
        "concavity error": [0.02, 0.025, 0.01],
        "concave points error": [0.01, 0.015, 0.008],
        "symmetry error": [0.02, 0.03, 0.015],
        "fractal dimension error": [0.003, 0.004, 0.002],
        "worst radius": [16.0, 25.0, 14.5],
        "worst texture": [25.0, 40.0, 20.0],
        "worst perimeter": [110.0, 160.0, 95.0],
        "worst area": [800.0, 1500.0, 600.0],
        "worst smoothness": [0.15, 0.2, 0.12],
        "worst compactness": [0.25, 0.3, 0.2],
        "worst concavity": [0.15, 0.2, 0.1],
        "worst concave points": [0.07, 0.1, 0.05],
        "worst symmetry": [0.3, 0.4, 0.25],
        "worst fractal dimension": [0.08, 0.1, 0.07],
    })

In [ ]:
# Recupera il modello in produzione
model_uri = f"models:/{MODEL_NAME}/Production"
model = mlflow.pyfunc.load_model(model_uri)


# Esegui inferenza
predictions = model.predict(new_data)

print("Inferenza:")
print(predictions)

Per fare pratica:
- Utilizzare un dataset differente, un dataset custom o un altro pubblico tipo "wine quality", basta che sia un problema di classificazione. Eventualmente, serve modificare il parametro "example_name", la funzione "load_dataset" e modificare i "new_data" per la parte di inferenza.
- Sperimentare più combinazioni di parametri/modelli.
- Aggiungere metriche di valutazione del modello (e loggarle su mlflow).
- Parametrizzare la logica di split train/test (e loggarla come parametro su mlflow).
- Cambiare di stato i modelli nel model registry sulla base di logiche a propria scelta.